<a href="https://colab.research.google.com/github/Yash1289/CoronaVirus_Tweet_Sentiment/blob/main/Coronavirus_Tweet_Sentiment_Analysis_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><u> Project Title : Sentiment Analysis : Predicting sentiment of COVID-19 tweets</u></b>

## <b> Problem Description </b>

### This challenge asks you to build a classification model to predict the sentiment of COVID-19 tweets.The tweets have been pulled from Twitter and manual tagging has been done then.

### The names and usernames have been given codes to avoid any privacy concerns.

### You are given the following information:
1. Location
2. Tweet At
3. Original Tweet
4. Label

In [1]:
import numpy as np
import pandas as pd

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
path = "/content/drive/MyDrive/Coronavirus Tweets.csv"
corona_df = pd.read_csv(path , engine = "python")

In [43]:
corona_df

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
...,...,...,...,...,...,...
41152,44951,89903,"Wellington City, New Zealand",14-04-2020,Airline pilots offering to stock supermarket s...,Neutral
41153,44952,89904,NaN,14-04-2020,Response to complaint not provided citing COVI...,Extremely Negative
41154,44953,89905,NaN,14-04-2020,You know its getting tough when @KameronWilds...,Positive
41155,44954,89906,NaN,14-04-2020,Is it wrong that the smell of hand sanitizer i...,Neutral


In [44]:
corona_df[corona_df["Location"] == "london"]

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
623,4422,49374,london,16-03-2020,Either #coronavirus will kill us or we will st...,Extremely Negative
5224,9023,53975,london,18-03-2020,PLEASE STOP PANIC BUYING! #COVID19 #coronaviru...,Negative
8511,12310,57262,london,19-03-2020,How are we supposed to avoid crowded places wh...,Negative
9141,12940,57892,london,20-03-2020,Supermarket sweep ... or not !!! #shopsempty #...,Neutral
11880,15679,60631,london,20-03-2020,225% increase in online grocery shopping As ov...,Positive
11881,15680,60632,london,20-03-2020,Waitrose short of staff in crisis as demand sp...,Negative
11986,15785,60737,london,20-03-2020,Local cafe serves as Portuguese grocery store ...,Positive
19854,23653,68605,london,24-03-2020,church street food &amp; wine have put prices ...,Extremely Negative
28441,32240,77192,london,04-04-2020,COVID-19: Kind landlord helps tenants stock fo...,Extremely Positive
29189,32988,77940,london,05-04-2020,Maybe we need to be a bit like Switzerland 20...,Positive


In [47]:
corona_df["Location"] = corona_df["Location"].str.capitalize()
corona_df["Location"] = corona_df["Location"].str.strip()

In [48]:
corona_df[corona_df["Location"] == "london"]

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment


In [49]:
import re

In [50]:
def cityFinder(location):
  if isinstance(location, str):
    cityAndOthers = re.split(r"-|,|/", location)
    return cityAndOthers[0]
  else:
    return location

In [51]:
corona_df["Location"] = corona_df["Location"].apply(cityFinder)

In [52]:
corona_df[corona_df["Location"] == "London, England"]

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment


In [63]:
values1 = pd.DataFrame(corona_df["Location"].value_counts())

In [64]:
modified_values1 = values1.reset_index().rename(columns = { "index" : "location" , "Location" : "value"})

In [65]:
modified_values1[modified_values1["location"] == "London "]

,location,value


In [62]:
corona_df["Location"] = corona_df["Location"].str.strip()

In [66]:
modified_values1

,location,value
0,London,1313
1,New york,727
2,United states,535
3,Washington,509
4,Los angeles,440
...,...,...
8617,New zealand (most of the time),1
8618,Sydney australia.,1
8619,Uk citizen of the world,1
8620,Bradwell abbey,1


In [67]:
corona_df["Sentiment"].value_counts()

Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

In [68]:
neutral_tweets = corona_df[corona_df["Sentiment"] == "Neutral"].index
corona_df.drop(neutral_tweets , inplace = True)

In [69]:
def Sentiment_Grouping(sentiment):
  if sentiment == "Extremely Positive":
    return "Positive"
  elif sentiment == "Extremely Negative":
    return "Negative"
  else:
    return sentiment

In [70]:
corona_df["Sentiment"] = corona_df["Sentiment"].apply(Sentiment_Grouping)

In [71]:
corona_df["Sentiment"].value_counts()

Positive    18046
Negative    15398
Name: Sentiment, dtype: int64

In [72]:
sentiment_grouping = corona_df.groupby(["Location" , "Sentiment"]).agg({ "OriginalTweet" : "count"}).reset_index()
sentiment_pivot = sentiment_grouping.pivot(index = "Location" , columns="Sentiment" , values="OriginalTweet")
sentiment_pivot

Sentiment,Negative,Positive
Location,,
,2.0,2.0
"""..i had to find the passage back to the place i was before..\r\r\nyou can check",NaN,1.0
"""deviantart""",1.0,NaN
#accessibility #justice #hugs #hope,NaN,1.0
#africa,3.0,NaN
...,...,...
Üt: 51.4761159,NaN,1.0
Üt: 51.560275,NaN,1.0
Üt: 51.56353,2.0,NaN


In [73]:
sentiment_pivot = sentiment_pivot.apply(lambda x: round(x*100/sum(x),2) if sum(x) > 10 else (x/x)*50, axis=1)
sentiment_pivot

Sentiment,Negative,Positive
Location,,
,50.0,50.0
"""..i had to find the passage back to the place i was before..\r\r\nyou can check",NaN,50.0
"""deviantart""",50.0,NaN
#accessibility #justice #hugs #hope,NaN,50.0
#africa,50.0,NaN
...,...,...
Üt: 51.4761159,NaN,50.0
Üt: 51.560275,NaN,50.0
Üt: 51.56353,50.0,NaN


In [90]:
sentiment_pivot.fillna(50.0 , inplace = True)

In [91]:
sentiment_pivot

Sentiment,Negative,Positive
Location,,
,50.0,50.0
"""..i had to find the passage back to the place i was before..\r\r\nyou can check",50.0,50.0
"""deviantart""",50.0,50.0
#accessibility #justice #hugs #hope,50.0,50.0
#africa,50.0,50.0
...,...,...
Üt: 51.4761159,50.0,50.0
Üt: 51.560275,50.0,50.0
Üt: 51.56353,50.0,50.0


In [78]:
sentiment_pivot.columns

Index(['Negative', 'Positive'], dtype='object', name='Sentiment')

In [92]:
sentimentBackup = sentiment_pivot.copy()

In [80]:
def Location_Cat(positivePercent):
  if positivePercent <= 55.0 and positivePercent >=45.0 :
    return 0
  elif positivePercent <= 70.0 and positivePercent >55.0 :
    return 1
  elif positivePercent <= 100.0 and positivePercent > 70 :
    return 2
  elif positivePercent < 45.0 and positivePercent >= 30 :
    return -1
  elif positivePercent < 30.0 and positivePercent >= 0 :
    return -2

In [97]:
sentimentBackup = sentimentBackup.reset_index().rename_axis(None, axis=1)

In [98]:
sentimentBackup["LocationCat"] = sentimentBackup["Positive"].apply(Location_Cat)

In [99]:
sentimentBackup

,Location,Negative,Positive,LocationCat
0,,50.0,50.0,0
1,"""..i had to find the passage back to the place...",50.0,50.0,0
2,"""deviantart""",50.0,50.0,0
3,#accessibility #justice #hugs #hope,50.0,50.0,0
4,#africa,50.0,50.0,0
...,...,...,...,...
7450,Üt: 51.4761159,50.0,50.0,0
7451,Üt: 51.560275,50.0,50.0,0
7452,Üt: 51.56353,50.0,50.0,0
7453,Üt: 54.975455,50.0,50.0,0


In [77]:
corona_df.columns

Index(['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet',
       'Sentiment'],
      dtype='object')

In [101]:
sentimentBackup.shape

(7455, 4)

In [103]:
corona_df.shape

(33444, 6)

In [104]:
corona_new_df = pd.merge(corona_df , sentimentBackup , how = "left" , on = "Location")

In [106]:
corona_new_df["Location"].fillna("Unknown" , inplace = True)
corona_new_df["LocationCat"].fillna(0 , inplace = True)

In [108]:
corona_new_df.drop(columns = ["Negative" , "Positive"] , axis = 1 , inplace = True)

In [109]:
corona_new_df

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,LocationCat
0,3800,48752,Uk,16-03-2020,advice Talk to your neighbours family to excha...,Positive,0.0
1,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive,0.0
2,3802,48754,Unknown,16-03-2020,My food stock is not the only one which is emp...,Positive,0.0
3,3803,48755,Unknown,16-03-2020,"Me, ready to go at supermarket during the #COV...",Negative,0.0
4,3804,48756,Üt: 36.319708,16-03-2020,As news of the regions first confirmed COVID-...,Positive,0.0
...,...,...,...,...,...,...,...
33439,44949,89901,Ohio,14-04-2020,I never that wed be in a situation &amp; worl...,Positive,1.0
33440,44950,89902,Unknown,14-04-2020,@MrSilverScott you are definitely my man. I fe...,Positive,0.0
33441,44952,89904,Unknown,14-04-2020,Response to complaint not provided citing COVI...,Negative,0.0
33442,44953,89905,Unknown,14-04-2020,You know its getting tough when @KameronWilds...,Positive,0.0


In [111]:
pd.DataFrame(corona_new_df.dtypes).rename(columns = {0:'dtype'})

,dtype
UserName,int64
ScreenName,int64
Location,object
TweetAt,object
OriginalTweet,object
Sentiment,object
LocationCat,float64


In [112]:
corona_new_df["TweetAt"] = pd.to_datetime(corona_new_df["TweetAt"])

In [115]:
corona_new_df["Month"] = corona_new_df["TweetAt"].apply(lambda x : x.month)
corona_new_df["Year"] = corona_new_df["TweetAt"].apply(lambda x : x.year)

In [116]:
corona_new_df

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,LocationCat,Month,Year
0,3800,48752,Uk,2020-03-16,advice Talk to your neighbours family to excha...,Positive,0.0,3,2020
1,3801,48753,Vagabonds,2020-03-16,Coronavirus Australia: Woolworths to give elde...,Positive,0.0,3,2020
2,3802,48754,Unknown,2020-03-16,My food stock is not the only one which is emp...,Positive,0.0,3,2020
3,3803,48755,Unknown,2020-03-16,"Me, ready to go at supermarket during the #COV...",Negative,0.0,3,2020
4,3804,48756,Üt: 36.319708,2020-03-16,As news of the regions first confirmed COVID-...,Positive,0.0,3,2020
...,...,...,...,...,...,...,...,...,...
33439,44949,89901,Ohio,2020-04-14,I never that wed be in a situation &amp; worl...,Positive,1.0,4,2020
33440,44950,89902,Unknown,2020-04-14,@MrSilverScott you are definitely my man. I fe...,Positive,0.0,4,2020
33441,44952,89904,Unknown,2020-04-14,Response to complaint not provided citing COVI...,Negative,0.0,4,2020
33442,44953,89905,Unknown,2020-04-14,You know its getting tough when @KameronWilds...,Positive,0.0,4,2020


##**EDA Section**

In [117]:
corona_eda_df = corona_new_df.copy()

In [120]:
#Importing the seaborn and matplotlib library for our data visualization tasks
import matplotlib.pyplot as plt
import seaborn as sns

In [119]:
#Setting different parameters of matplotlib and seaborn for better visualization 
plt.rcParams['figure.figsize'] = 10,7
sns.set_context('paper', font_scale=1.4)
sns.set_theme(style='darkgrid')

In [ ]:
sns.barplot